In [ ]:
import yaml
import json
import vocab_build_tools as tools

stream = open('./config.yaml', 'r')

documents = yaml.load_all(stream, Loader=yaml.FullLoader)

for doc in documents:
    merged_df = tools.create_df(doc['termLists'])
    config = doc

examples = merged_df[~merged_df['examples'].isna()]

classes = ['TaxonConcept', 'TaxonConceptMappings', 'TaxonName', 'NomenclaturalType']

with open('../docs/examples/README.md', 'w') as f:
    
    f.write("# Examples\n\n")
    
    for cl in classes:
        f.write("\n## " + cl + "\n\n")
        
        for index, row in examples[examples['organizedInClass'] == 'http://rs.tdwg.org/tcs/terms/' + cl].iterrows():
            alias = [n['vann_preferredNamespacePrefix'] for n in config['termLists'] if n['vann_preferredNamespaceUri'] == row['namespace']][0]
            f.write("\n### " + alias + ":" + row['localName'] + "\n\n") 
            
            if type(row['examples']) == list:
                for ex in row['examples']:
                    text = tools.add_example(ex)
                    f.write(text)


TaxonConcept-example-1

```turtle
[] a tcs:TaxonConcept ;
    dcterms:title "Dicranoloma blumei sec. Klazenga 1999" ;
    tcs:accordingTo <https://www.tropicos.org/reference/9020903> 
    tcs:taxonName <https://www.tropicos.org/name/35121475> .

<https://www.tropicos.org/name/35121475> a tcs:TaxonName ;
    tcs:taxonNameString "Dicranoloma blumei" ;
    dwc:scientificNameAuthorship "(Nees) Renauld" .

<https://www.tropicos.org/reference/9020903> a bibo:AcademicArticle ;
    dcterms:bibliographicCitation """Klazenga, N. (1999). A revision of the 
            Malesian species of Dicranoloma (Dicranaceae, Musci). Journal of the 
            Hattori Botanical Laboratory 87: 1-130.""" .
```

[&lsqb;TaxonConcept-example-1.ttl&rsqb;](https://github.com/tdwg/tcs2/blob/master/examples/TaxonConcept-example-1.ttl)&nbsp;[&lsqb;TaxonConcept-example-1.jsonld&rsqb;](https://github.com/tdwg/tcs2/blob/master/examples/TaxonConcept-example-1.jsonld)


TaxonConcept-example-2

```turtle
[] a tcs:TaxonConce